In [1]:
%load_ext autoreload
%autoreload 2

# WORKSHOP - LLM-based Agents 

In [2]:
USER_INSTRUCTIONS = "Write some tests for the chat.py module'"
# USER_INSTRUCTIONS = "Document the code found inside the file 'thread.py'"
# USER_INSTRUCTIONS = "Suggest some code improvements for the codebase.py module'"

### Planner agent

In [3]:
planner_prompt = """
You are an LLM agent specialized in writing technical plans for a given task.
The steps you write in your plans will be executed by other LLM agents which have to interact with the file system of the user.
You should therefore consider steps such as listing files found inside a directory, reading files, writing files, etc.

IMPORTANT NOTES:
You plan should be as concised as possible, using as little steps as possible and only containing the necessary information.
Try not to exceed 4-5 steps.
Do not try and execute that plan, just write it down.

Plan the following task:
"""

In [5]:
from codeas.thread import Thread

# build agent
planner = Thread(system_prompt=planner_prompt, model="gpt-4o", use_console=False)

# add instructions
planner.add_message({"role": "user", "content": USER_INSTRUCTIONS})

# run agent
plan = planner.run()

1. **List Files in Directory**: List all files in the directory to locate `chat.py`.
2. **Read chat.py**: Read the contents of `chat.py` to understand its functions and classes.
3. **Create Test File**: Create a new file named `test_chat.py` in the same directory.
4. **Write Tests**: Write unit tests for the functions and classes found in `chat.py` and save them in `test_chat.py`.
5. **Verify Test File**: Ensure `test_chat.py` is correctly written and saved.

### Tools

In [6]:
from codeas import tools

In [7]:
# format dict output for better readability
import json
def print_dict(d):
    print(json.dumps(d, indent=4))

In [8]:
for schema in tools.get_schemas([tools.list_files, tools.read_file, tools.create_file]):
    print_dict(schema)

{
    "type": "function",
    "function": {
        "name": "list_files",
        "description": "list all of the files in a given directory",
        "parameters": {
            "properties": {
                "dir_path": {
                    "description": "relative directory path",
                    "type": "string"
                }
            },
            "required": [
                "dir_path"
            ],
            "type": "object"
        }
    }
}
{
    "type": "function",
    "function": {
        "name": "read_file",
        "description": "reads the content of a file",
        "parameters": {
            "properties": {
                "path": {
                    "description": "relative file path, including file name",
                    "type": "string"
                },
                "line_start": {
                    "default": 1,
                    "description": "start line to read",
                    "type": "integer"
                },
         

### Engineer agent

In [10]:
engineer_prompt = """
You are a senior software engineer who needs to fulfill a technical task.
You will also be provided with a technical plan that has been designed to fulfill this task.
You can interact with the file system using the tools made available to you such as listing, reading and writing files.
Only use these tools when necessary.

Here is the technical task and plan to follow:
""".strip()

In [11]:
from codeas.thread import Thread

# build agent
engineer = Thread(system_prompt=engineer_prompt, model="gpt-4o", use_console=False,
    tools=[tools.list_files, tools.read_file, tools.create_file]
)
# add instructions
engineer.add_message({"role": "user", "content": f"TASK:\n\n{USER_INSTRUCTIONS}"})

# add plan to the conversation
engineer.add_message({"role": "user", "content": f"PLAN:\n\n{plan['content']}"})

# run agent
action1 = engineer.run()

{
│   'role': 'assistant',
│   'content': None,
│   'tool_calls': [
│   │   {
│   │   │   'id': 'call_vd4M4nC8OD9rLrgZk7hXhlpC',
│   │   │   'type': 'function',
│   │   │   'function': {
│   │   │   │   'name': 'list_files',
│   │   │   │   'arguments': '{"dir_path":"./"}'
│   │   │   }
│   │   }
│   ]
}

In [13]:
for call in engineer.run_calls(action1["tool_calls"]):
    print(call["output"])

Calling: list_files({"dir_path":"./"})
>>> Call successful

['tests/test_codebase.py', 'tests/test_request.py', 'tests/utils.py', 'tests/test_assistant.py', 'examples/ml_repo_s/src/ml_pipeline.py', 'examples/ml_repo_xs/src/predictor.py', 'examples/ml_repo_xs/src/pipeline.py', 'examples/java_repo_s/src/MainApplication.java', 'examples/java_repo_s/src/StringUtil.java', 'examples/java_repo_s/src/UserEntity.java', 'examples/js_repo_s/src/ClassA.js', 'examples/js_repo_s/src/ClassC.js', 'examples/js_repo_s/src/ClassB.js', 'examples/ml_repo_m/main.py', 'examples/ml_repo_m/src/predictor.py', 'examples/ml_repo_m/src/processor.py', 'examples/ml_repo_m/src/evaluator.py', 'debug/dd.py', 'src/codeas/thread.py', 'src/codeas/configs.py', 'src/codeas/terminal.py', 'src/codeas/tools.py', 'src/codeas/codebase.py', 'src/codeas/agents.py', 'src/codeas/chat.py', 'src/codeas/utils.py', 'src/codeas/commands.py', 'src/codeag/pseudocode.py', 'src/codeag/core/agents/planner.py', 'src/codeag/core/agents/parser.p

In [14]:
# add action to the conversation
engineer.add_message(action1)

In [15]:
# add action output to the conversation
engineer.add_tool_responses(action1["tool_calls"])

Calling: list_files({"dir_path":"./"})
>>> Call successful



In [16]:
# view execution trajectory
engineer._messages

[{'role': 'system',
  'content': 'You are a senior software engineer who needs to fulfill a technical task.\nYou will also be provided with a technical plan that has been designed to fulfill this task.\nYou can interact with the file system using the tools made available to you such as listing, reading and writing files.\nOnly use these tools when necessary.\n\nHere is the technical task and plan to follow:'},
 {'role': 'user',
  'content': "TASK:\n\nWrite some tests for the chat.py module'"},
 {'role': 'user',
  'content': 'PLAN:\n\n1. **List Files in Directory**: List all files in the directory to locate `chat.py`.\n2. **Read chat.py**: Read the contents of `chat.py` to understand its functions and classes.\n3. **Create Test File**: Create a new file named `test_chat.py` in the same directory.\n4. **Write Tests**: Write unit tests for the functions and classes found in `chat.py` and save them in `test_chat.py`.\n5. **Verify Test File**: Ensure `test_chat.py` is correctly written and 

# Reasoning

In [17]:
reasoner_prompt = """
You are an LLM agent specialized in reasoning about the actions taken by other LLM agents in order to fulfill a given task.

Here is the task at hand, the plan to follow and the current execution trajectory:
"""

In [18]:
# build agent
reasoner = Thread(system_prompt=reasoner_prompt, model="gpt-4o", use_console=False)
# add execution trajectory
reasoner._messages.extend(engineer._messages[1:])

In [19]:
# add instructions
reasoner_instructions = """
Write some observation of the last action taken by the agent and what the next action should be. 
Format it as ### Observation\n[your text here] ### Proposed next action\n[your text here].
If the full request seems to have been fulfilled, return ### FINISHED.
"""
reasoner.add_message({"role": "user", "content": reasoner_instructions})

In [20]:
# run agent
observation1 = reasoner.run()

### Observation
The agent successfully listed all files in the directory and located `chat.py` in the `src/codeas` directory.

### Proposed next action
Read the contents of `src/codeas/chat.py` to understand its functions and classes.

In [21]:
engineer.add_message(observation1)

In [22]:
# view trajectory
engineer._messages

[{'role': 'system',
  'content': 'You are a senior software engineer who needs to fulfill a technical task.\nYou will also be provided with a technical plan that has been designed to fulfill this task.\nYou can interact with the file system using the tools made available to you such as listing, reading and writing files.\nOnly use these tools when necessary.\n\nHere is the technical task and plan to follow:'},
 {'role': 'user',
  'content': "TASK:\n\nWrite some tests for the chat.py module'"},
 {'role': 'user',
  'content': 'PLAN:\n\n1. **List Files in Directory**: List all files in the directory to locate `chat.py`.\n2. **Read chat.py**: Read the contents of `chat.py` to understand its functions and classes.\n3. **Create Test File**: Create a new file named `test_chat.py` in the same directory.\n4. **Write Tests**: Write unit tests for the functions and classes found in `chat.py` and save them in `test_chat.py`.\n5. **Verify Test File**: Ensure `test_chat.py` is correctly written and 

In [23]:
# execute next action
action2 = engineer.run()

{
│   'role': 'assistant',
│   'content': None,
│   'tool_calls': [
│   │   {
│   │   │   'id': 'call_TVcfT0s3OLOpiJiAJTz1xFky',
│   │   │   'type': 'function',
│   │   │   'function': {
│   │   │   │   'name': 'read_file',
│   │   │   │   'arguments': '{"path":"src/codeas/chat.py"}'
│   │   │   }
│   │   }
│   ]
}

# Reflection

In [24]:
reflection_prompt = """
You are an LLM agent specialized in reflecting on the actions taken by other LLM agents given some observations.
Here is the observation and action taken as a result of it:
"""

In [25]:
# build agent
reflection = Thread(system_prompt=reflection_prompt, model="gpt-4o", use_console=False,
    tools=[tools.read_file] # workaround only to add the tool response to the thread
)
# add observation, action and tool responses to the conversation
reflection.add_message(observation1)
reflection.add_message(action2)
reflection.add_tool_responses(action2["tool_calls"])

Calling: read_file({"path":"src/codeas/chat.py"})
>>> Call successful



In [26]:
# add agent instructions
reflection_instructions = """
Return some feedback on the action taken by the agent in the following format:
### Correct action taken: \n yes/no
### Feedback: \n [your feedback here]
""".strip()
reflection.add_message({"role": "user","content": reflection_instructions})

# run agent
feedback1 = reflection.run()

### Correct action taken:
Yes

### Feedback:
The agent correctly identified the need to read the contents of `src/codeas/chat.py` to understand its functions and classes. This action is essential for gaining insights into the structure and functionality of the `Chat` class, which includes methods for handling messages, running agents, and executing commands. The information retrieved will be valuable for any subsequent tasks that involve interacting with or modifying the `Chat` class.

### Full application (PSEUDOCODE)

In [ ]:
# define a plan for the task
plan = planner.run()

# iterate over the plan's steps until the task is finished
FINISHED = False
while FINISHED is False:
    # execute the plan's steps
    action = engineer.run()
    # check that the action is correct
    feedback = reflection.run()
    if feedback == "positive":
        # if feedback is positive, reason about what the next step should be
        observation = reasoner.run()
        if observation == "finished":
            # if no more steps are needed, finish the task
            FINISHED = True
        else:
            # if more steps are needed, add the observation and run the next step
            engineer.add_message(observation)
    else:
        # if feedback is negative, add the feedback and run the same step again
        engineer.add_message(feedback)